In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K

config = tf.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
K.set_session(sess)  # set this TensorFlow session as the default session for Keras

In [2]:
# load data
idx_features_labels = np.genfromtxt("./data/cora.content", dtype=np.dtype(str))
features = np.array(idx_features_labels[:, 1:-1], dtype=np.float32)

In [3]:
def encode_label(labels):
    classes = set(labels)
    # 生成class对应的向量的字典
    classes_dict = {c: np.eye(len(classes))[index, :] for index, c in enumerate(classes)}
    # 生成向量
    label_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return label_onehot, classes_dict
labels_list = idx_features_labels[:, -1]
labels, labels_class = encode_label(labels_list)

In [4]:
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
# get node:index dict 
idx_map = {node: index for index, node in enumerate(idx)}

In [7]:
# get edges
edges_unordered = np.genfromtxt("./data/cora.cites", dtype=np.int32)
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)

In [8]:
# get adj matrix
adj = np.zeros((labels.shape[0], labels.shape[0]))
for edge in edges:
    adj[edge[0], edge[1]] = 1.
    
adj = adj + adj.T  - np.diagflat(np.diag(adj))

In [9]:
print('Dataset has {} nodes, {} edges, {} features.'.format(adj.shape[0], edges.shape[0], features.shape[1]))

Dataset has 2708 nodes, 5429 edges, 1433 features.


In [11]:
X = features
A = adj
y = labels
X.shape, A.shape, y.shape

((2708, 1433), (2708, 2708), (2708, 7))

In [12]:
# normalize X
X = X / np.sum(X, axis=1).reshape((-1, 1))

In [13]:
# custom function
def normalize_adj(adj, symmetric=True):
    # D^(-1/2) * A * D^(-1/2)
    if symmetric:
        D = np.sum(adj, axis=1)
        D = np.power(D, -1/2) 
        D = np.diagflat(D.flatten(), 0) 
        #  D^(-1/2) * A * D^(-1/2)
        a_norm = D * adj * D
    else:
        # D^(-1) * A
        D = np.diagflat(np.power(np.sum(adj, axis=1), -1).flatten(), 0) 
        a_norm = D * adj
    return a_norm

In [14]:
FILTER = 'chebyshev'  # 'chebyshev'
SYM_NORM = True  # symmetric (True) vs. left-only (False) normalization
MAX_DEGREE = 2  # maximum polynomial degree

if FILTER == "localpool":
    A = A + np.eye(A.shape[0])
    A_norm = normalize_adj(A, symmetric=SYM_NORM) 
    support = 1 
    
    graph = [X, A_norm]
    G = [keras.layers.Input(shape=(None,))]

elif FILTER == "chebyshev":
    A_norm = normalize_adj(A, symmetric=True)
    laplacian = np.eye(A_norm.shape[0]) - A_norm
    try:
        largest_eigenvalues = sp.linalg.eigsh(laplacian, k=1, which="LM", return_eigenvectors=False)[0]
    except Exception as e:
        largest_eigenvalues = 2
    
    rescaled_laplacian = (2./largest_eigenvalues)*laplacian - np.eye(laplacian.shape[0])
    
    T_k = []
    T_k.append(np.eye(rescaled_laplacian.shape[0])) # T(0) = 1
    T_k.append(rescaled_laplacian)  # T(1) = x
    def chebyshev_recurrence(T_k_minus_one, T_k_minus_two, rescaled_laplacian):
        """
        2 * x * T(k-1)(x) - T(k-2)(x), T(0) = 1, T(1) = x
        :param T_k_minus_one: T(k-1)(X)
        :param T_k_minus_two: T(k-2)(X)
        :param X: X
        :return: Tk(X)
        """
        # Tk(X) = 2X * T(k-1)(X) - T(k-2)(X)
        return 2 * rescaled_laplacian * T_k_minus_one - T_k_minus_two
    
    for i in range(2, MAX_DEGREE + 1):
        T_k.append(chebyshev_recurrence(T_k[-1], T_k[-2], rescaled_laplacian))
        
    support = MAX_DEGREE + 1
    graph = [X] + T_k
    G = [keras.layers.Input(shape=(rescaled_laplacian.shape[0],)) for _ in range(support)]
    
else:
    raise Exception('Invalid filter type.')

In [15]:
class GCN(keras.layers.Layer):
    def __init__(self,
                 units,
                 supports=1,        
                 activation=None,   
                 use_bias=True,     
                 kernel_initializer="glorot_uniform",
                 bias_initializer="zeros",
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.units = units  
        self.activation = keras.activations.get(activation)
        self.use_bias = use_bias    

        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.bias_initializer = keras.initializers.get(bias_initializer)

        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = keras.regularizers.get(bias_regularizer)

        self.kernel_constraint = keras.constraints.get(kernel_constraint)
        self.bias_constraint = keras.constraints.get(bias_constraint)

        self.supports_masking = True    
        self.supports = supports    

        assert self.supports >= 1   


    def build(self, input_shape):
        """
        Y = GraphConvolution(y.shape[1], support, activation='softmax')([H] + G)
        :param input_shape:
        """
        features_shape = tf.TensorShape(input_shape[0]).as_list()
        assert len(features_shape) == 2     # (batch_size, feature_dim)

        input_dim = features_shape[1]
        self.kernel = self.add_weight(shape=(input_dim * self.supports, self.units),
                                      initializer=self.kernel_initializer,
                                      name="kernel",
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name="bias",
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None

        self.built = True

    def call(self, inputs, mask=None):
        """
        Y = GraphConvolution(y.shape[1], support, activation='softmax')([H] + G)
        """
        features = inputs[0]    # (2708, 1433)  => (node_num, feature_dim)
        A = inputs[1:]      # [(2708, 2708), ...] => [(node_num, node_num), ...]

        supports = []
        for i in range(self.supports):
            supports.append(K.dot(A[i], features))   # (node_num, node_num) * (node_num, feature_dim) => (node_num, feature_dim)

        supports = K.concatenate(supports, axis=1)      # (node_num, feature_dim * self.supports)
        outputs = K.dot(supports, self.kernel)
        if self.use_bias:
            outputs = outputs + self.bias
        return self.activation(outputs)

    def compute_output_shape(self, input_shape):
        return input_shape[0][0], self.units

In [16]:
# build model 
X_in = keras.layers.Input(shape=(X.shape[1],))
H = keras.layers.Dropout(0.5)(X_in)
H = GCN(16, support, activation="relu", kernel_regularizer=keras.regularizers.l2(5e-4))([X_in] + G)
H = keras.layers.Dropout(0.5)(H)
Y = GCN(y.shape[1], support, activation="softmax")([H] + G)
model = keras.models.Model(inputs=[X_in] + G, outputs=Y)

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2708)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2708)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 2708)         0                                            
__________________________________________________________________________________________________
gcn (GCN) 

In [18]:
model.compile(loss='categorical_crossentropy', optimizer="sgd")

ValueError: Argument must be a dense tensor: [[None, None, None, None], None, None, None] - got shape [4], but wanted [4, 4].